In [48]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup
import sys
import os

import sklearn
from sklearn import cross_validation
from sklearn import svm
from sklearn import metrics

reload(sys)
#sys.setdefaultencoding("ISO-8859-1")
sys.setdefaultencoding("UTF-8")

def clean_text(raw_text):
    review_text = BeautifulSoup(raw_text).get_text()
    words = review_text.lower().split()
    return(" ".join(words))

In [49]:
import re 

languages = ["english"]
datafiles = ["summary-english-truth.txt"]
tasks = ["age"]

train = pd.read_csv("summary-english-truth.txt", header=0, delimiter="\t", quoting=1)
num_text = train["text"].size

clean_train_data = []
urls = []
hashtags = []
for i in xrange( 0, num_text):
    one_clean_line = clean_text( train["text"][i] )
    new_clean_line = ""
    #replacing links
    url = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', one_clean_line)
    for one_url in url:
        new_clean_line = one_clean_line.replace(one_url, " LINK_HERE ")
        one_clean_line = new_clean_line
    urls.append(url)
    
    hashtag = re.findall('#(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', one_clean_line)
    
    for one_hashtag in hashtag:
        new_clean_line = one_clean_line.replace(one_hashtag, " HASHTAG_HERE ")
        one_clean_line = new_clean_line
        
    urls.append(url)
    hashtags.append(hashtag)
    clean_train_data.append( one_clean_line )

In [50]:
vectorizer = TfidfVectorizer(analyzer = "word", tokenizer = None, preprocessor = None)
train_x = vectorizer.fit_transform(clean_train_data)
train_x = train_x.toarray()
train_x.shape

(152, 18457)

In [51]:
def makeArff(train_x, train_y, task, feature_names):
    rows, cols = train_x.shape
    file_name = "new-"+task + ".arff"
    file = open(file_name, "w")
    file.write("@relation" + " " + task + "\n")
    
    for each_feature_name in feature_names:
        file.write("@attribute" + " " + each_feature_name + " " + "real\n")
    
    if task == "gender":
        class_vals = "{0,1}"
    else:
        class_vals = "{0,1,2,3}"
    
    file.write("@attribute" + " target_class " + class_vals + "\n")
    file.write("@data"+ "\n")
    
    for row in range(0,rows):
        for col in range(0,cols):
            file.write(str(train_x[row][col]) + ",")
        file.write(str(train_y[row]) + "\n")
    file.close()

In [52]:
feature_names = vectorizer.get_feature_names()

In [53]:
tasks = ["age", "gender"]


In [54]:
for task in tasks:
    train_y = train[task]
    makeArff(train_x, train_y, task, feature_names)

In [55]:
#looking into age - 9000 features

In [57]:
with open('new-age-important-words-using-info-gain.txt') as f:
    alist = [line.rstrip() for line in f]
all_indices_ranked = alist[0].split(',')
all_indices_ranked = [int(x) for x in all_indices_ranked]
all_indices_ranked = [x-1 for x in all_indices_ranked]

num_features = 9000
xx = [all_indices_ranked[x] for x in range(0, num_features)]
xx = tuple(xx)
age_features_smaller_train_x = train_x[:, xx]

In [60]:
train = pd.read_csv("summary-english-truth.txt", header=0, delimiter="\t", quoting=1)
train_y_age = train['age']
clf1 = svm.SVC(kernel='poly', degree=3, coef0=10, gamma=1)
scores_age = cross_validation.cross_val_score(clf1, age_features_smaller_train_x, train_y_age, cv=10, scoring='accuracy')

In [61]:
scores_age.mean()

0.80515406162464986

In [63]:
with open('new-gender-important-words-using-info-gain.txt') as f:
    alist = [line.rstrip() for line in f]
all_indices_ranked = alist[0].split(',')
all_indices_ranked = [int(x) for x in all_indices_ranked]
all_indices_ranked = [x-1 for x in all_indices_ranked]

num_features = 7000
xx = [all_indices_ranked[x] for x in range(0, num_features)]
xx = tuple(xx)
gender_features_smaller_train_x = train_x[:, xx]

In [64]:
train = pd.read_csv("summary-english-truth.txt", header=0, delimiter="\t", quoting=1)
train_y_gender = train['gender']
clf2 = svm.SVC(kernel='poly', degree=2, coef0=10000, gamma=1)
scores_gender = cross_validation.cross_val_score(clf2, gender_features_smaller_train_x, train_y_gender, cv=10, scoring='accuracy')

In [65]:
scores_gender.mean()

0.76875000000000004